Author: Gickel OKABI

Python developer

mail : i.gickelokabi@gmail.com

.

CAR SALE SCRAPING for Kijiji
We need a pyhton 3 script to get us cars for sale on the following sites:
Kijiji.ca to save  to CSV

-search limiters

•	SUV

•	2017 or newer

•	less than 80,000km

•	less than 1000km from L5B 0G5 (Ontario and Quebec on Kijiji but 

•	less than $25,000


KIJIJI
https://www.kijiji.ca/b-cars-trucks/ontario/suv/2017__/k0c174l9004a68?price=__25000&kilometers=__80000

Autotrader
https://www.autotrader.ca/cars/on/mississauga/?rcp=15&rcs=0&srt=4&yRng=2017%2C&pRng=%2C25000&oRng=%2C80000&prx=1000&prv=Ontario&loc=mississauga&body=SUV&hprc=True&wcp=True&sts=New-Used&inMarket=advancedSearch

Details to collect

1.	Page URL
2.	Make
3.	Model
4.	Mileage/Kilometers
5.	Status/condition (e.g. new/used)
6.	Transmission
7.	Description

In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import time
from itertools import cycle


In [2]:
headers = {
    'Referer': 'https://www.kijiji.ca/',
    'Sec-Ch-Ua': 'v="Chromium";v="114", " Not A;Brand";v="8", "Google Chrome";v="114"',

    'Sec-Ch-Ua-Mobile': '?0',
    'Sec-Ch-Ua-Platform': "Windows",
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    }
    

# 1 : Obtain proxies


First, we need to collect free proxies from the internet to avoid detection.

We will use selenium to load pages that support javascript

.

In [3]:
#Importing the necessary libraries


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC

chrome_options = Options()
chrome_options.add_argument("--headless") 
driver = webdriver.Chrome(options=chrome_options)

.

The links of the selected proxy sites

In [30]:
free_url = "http://free-proxy.cz/en/proxylist/country/all/https/ping/all"
free_proxy_url = "https://free-proxy-list.net/"
proxyscrape_url = 'https://proxyscrape.com/free-proxy-list'
hidemy_rul = 'https://hidemy.name/en/proxy-list/?type=s#list'

## Proxyscrape

In [5]:
driver.get(proxyscrape_url)

time.sleep(15)

In [6]:
proxyscrape_data = pd.read_html(driver.page_source)[0]

In [7]:
proxyscrape_data.shape

(400, 8)

In [8]:
proxyscrape_data.head(5)

,IP Address,Port,Protocol,Country,Anonymity,Latency,Uptime,Alive since
0,98.170.57.231,4145,Socks5,United States,Elite,3006ms,82.5%,3 hours
1,200.123.29.45,3128,HTTP,Peru,Anonymous,4588ms,31.9%,17 minutes
2,192.111.130.2,4145,Socks5,United States,Elite,870ms,73.7%,40 minutes
3,185.211.57.74,4002,HTTP,Iran,Anonymous,2949ms,99.6%,8 hours
4,222.88.167.22,9002,HTTP,China,Elite,1143ms,33.6%,16 minutes


.

## Free-proxy

In [9]:
free_proxy = requests.get(free_proxy_url,  headers=headers)

In [10]:
free_proxy_data = pd.read_html(free_proxy.text)[0]

In [11]:
free_proxy_data.shape

(300, 8)

In [12]:
free_proxy_data.head(5)

,IP Address,Port,Code,Country,Anonymity,Google,Https,Last Checked
0,123.202.159.108,80,HK,Hong Kong,anonymous,no,no,10 secs ago
1,178.33.3.163,8080,FR,France,elite proxy,no,yes,10 secs ago
2,58.27.59.249,80,MY,Malaysia,elite proxy,NaN,no,10 secs ago
3,103.216.103.163,80,HK,Hong Kong,anonymous,yes,no,10 secs ago
4,13.81.217.201,80,NL,Netherlands,elite proxy,NaN,no,10 secs ago


.

## Hidemy 

In [ ]:
hidemy_request = requests.get(hidemy_rul)

In [ ]:
hidemy_data = pd.read_html(hidemy_request.text)[0]

In [ ]:
hidemy_data.shape()

In [ ]:
hidemy_data.head()

.

This function takes a dataframe parameter and returns a list of proxies taking into account the ip address and the port

In [13]:
def prox_ip_liste(dataframe, ip_col, port_col):
    proxi_liste = []
    for i, row in dataframe.iterrows():
        proxi_line = {}
        proxi_line.update({'http': f'http://{row[ip_col]}:{row[port_col]}', 'https' : f'http://{row[ip_col]}:{row[port_col]}'})
        proxi_liste.append(proxi_line)   
    return proxi_liste  

.

This function takes a parameter the list of selected proxies, it tests the validity. then it returns the list of valid proxies

In [14]:
def test_proxy(proxy_liste):
    valide_proxies = []
    url = 'https://www.kijiji.ca/'
    i=0
    for proxy in proxy_liste:
        try:
            response = requests.get(url, proxies=proxy, headers=headers, timeout=5)
            if response.status_code == 200:
                print('Proxy valide:', proxy)
                valide_proxies.append(proxy)
                i = i+1
            else:
                print('Proxy invalide:', proxy)
        except requests.exceptions.RequestException as e:
            print('Proxy invalide:', 'ereur')
    print(i)
    return valide_proxies

.

In [15]:
free_proxy_frame = prox_ip_liste(free_proxy_data, "IP Address", "Port")
proxyscrape_frame = prox_ip_liste(free_proxy_data, "IP Address", "Port")

In [17]:
free_proxy_frame[:5]

[{'http': 'http://123.202.159.108:80', 'https': 'http://123.202.159.108:80'},
 {'http': 'http://178.33.3.163:8080', 'https': 'http://178.33.3.163:8080'},
 {'http': 'http://58.27.59.249:80', 'https': 'http://58.27.59.249:80'},
 {'http': 'http://103.216.103.163:80', 'https': 'http://103.216.103.163:80'},
 {'http': 'http://13.81.217.201:80', 'https': 'http://13.81.217.201:80'}]

In [18]:
proxyscrape_frame[:5]

[{'http': 'http://123.202.159.108:80', 'https': 'http://123.202.159.108:80'},
 {'http': 'http://178.33.3.163:8080', 'https': 'http://178.33.3.163:8080'},
 {'http': 'http://58.27.59.249:80', 'https': 'http://58.27.59.249:80'},
 {'http': 'http://103.216.103.163:80', 'https': 'http://103.216.103.163:80'},
 {'http': 'http://13.81.217.201:80', 'https': 'http://13.81.217.201:80'}]

In [19]:
proxy_liste = free_proxy_frame + proxyscrape_frame

.

We now have the list of our valid proxies


In [29]:
valide_proxies = test_proxy(proxy_liste)

Proxy valide: {'http': 'http://8.219.97.248:80', 'https': 'http://8.219.97.248:80'}
Proxy valide: {'http': 'http://18.143.215.49:80', 'https': 'http://18.143.215.49:80'}
Proxy valide: {'http': 'http://110.34.3.229:3128', 'https': 'http://110.34.3.229:3128'}
Proxy valide: {'http': 'http://20.44.206.138:80', 'https': 'http://20.44.206.138:80'}
Proxy valide: {'http': 'http://191.186.106.34:8080', 'https': 'http://191.186.106.34:8080'}
Proxy invalide: ereur
Proxy invalide: ereur
Proxy valide: {'http': 'http://18.143.215.49:80', 'https': 'http://18.143.215.49:80'}
Proxy invalide: ereur
Proxy valide: {'http': 'http://191.186.106.34:8080', 'https': 'http://191.186.106.34:8080'}
Proxy valide: {'http': 'http://186.121.235.222:8080', 'https': 'http://186.121.235.222:8080'}
8


In [22]:
valide_proxies[:5]

[{'http': 'http://202.46.145.4:8080', 'https': 'http://202.46.145.4:8080'},
 {'http': 'http://8.219.97.248:80', 'https': 'http://8.219.97.248:80'},
 {'http': 'http://113.53.231.133:3129', 'https': 'http://113.53.231.133:3129'},
 {'http': 'http://18.143.215.49:80', 'https': 'http://18.143.215.49:80'},
 {'http': 'http://110.34.3.229:3128', 'https': 'http://110.34.3.229:3128'}]

.

# 2. We will now focus on the sites in which we will scrap the data


site_url : https://www.kijiji.ca/

The site structure lets us know the number of pages, so we will choose the page range policy. Thus, we can indicate which pages to scrape


.

In [23]:
# This function takes as parameters the start page and the last page to scrape and returns all the pages
def get_page_lint(starting_age, ending_page):
    
    all_page_link = []
    
    for i in range(starting_age, 1+ending_page):
        link = f'https://www.kijiji.ca/b-cars-trucks/ontario/suv/2017__/page-{i}/k0c174l9004a68?price=__25000&kilometers=__80000'
        all_page_link.append(link)
        
    return all_page_link    
    

In [24]:
all_page_link = get_page_lint(1,2)

In [25]:
all_page_link

['https://www.kijiji.ca/b-cars-trucks/ontario/suv/2017__/page-1/k0c174l9004a68?price=__25000&kilometers=__80000',
 'https://www.kijiji.ca/b-cars-trucks/ontario/suv/2017__/page-2/k0c174l9004a68?price=__25000&kilometers=__80000']

.

In [26]:
# let's go through each page link to get all article links
def all_car_link(all_page_link, valide_proxies):
    all_car_links = []
    
    #Loop on the list of proxies
    proxy_cycle = cycle(valide_proxies)

    
    for page_link in all_page_link:
        proxy = next(proxy_cycle)
        try:     
            response = requests.get(page_link,proxies=proxy, headers=headers)
        except ProxyError as e:
            response = requests.get(page_link, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')

        all_care = soup.find_all('div', class_='search-item')

        all_car_links = ["".join(["https://www.kijiji.ca", care.find('div', class_ = "info").find('div', class_="title").find('a')['href']]) for care in all_care ]
  
            
    return all_car_links

.

In [ ]:
all_car_links = all_car_link(all_page_link, valide_proxies)

.

In [9]:
def data_collecte(all_car_link,valide_proxies):
    #Loop on the list of proxies
    proxy_cycle = cycle(valide_proxies)
    
    titles = []
    makes = []
    models = []
    kilometers = []
    conditions = []
    transmissions = []
    descriptions = []
    page_links = []
    
    for link in all_car_link:
        proxy = next(proxy_cycle)
        
        try:     
            car_response = requests.get(link, proxies=proxy, headers=headers)
        except ProxyError as e:
            response = requests.get(page_link, headers=headers)
            time.sleep(2)
        
        car_soup = BeautifulSoup(car_response.content, 'html.parser')
        overview = car_soup.find('div', class_="overview-2535594310").find('div', class_="sc-hHfuMS gnhTxo")
        
        try:
            title = car_soup.find_all('h1', class_="title-2323565163").text
        except:
            title =""
        try:    
            make = overview.find(lambda tag: tag.name and tag.text == 'Make: ').find_next_sibling().text
        except:
            make = ""
        try:    
            model = overview.find(lambda tag: tag.name and tag.text == 'Model: ').find_next_sibling().text
        except:
            model =""
            
        try:    
            kilometer = overview.find(lambda tag: tag.name and tag.text == 'Kilometers').find_next_sibling().text
        except:
            kilometer =""
        try:    
            condition = overview.find(lambda tag: tag.name and tag.text == 'Condition').find_next_sibling()
        except:
            condition = ""
        try:    
            transmission = overview.find(lambda tag: tag.name and tag.text == 'Transmission: ').find_next_sibling()
        except:
            transmission =""
        try:    
            description = car_soup.find('div', class_="autosDescriptionBox-2447469030").text
        
        except:
            description = ""
            
        # Save items to list
        titles.append(title)
        makes.append(make)
        models.append(model)
        kilometers.append(kilometer)
        conditions.append(condition)
        transmissions.append(transmission)
        descriptions.append(description)
        page_links.append(link)
        
        time.sleep(2)

        
    #creation of the data frame    
    dataframe = pd.DataFrame({
                'title' : titles,
                'make':makes,
                'model' : models,
                'kilometer' :kilometers,
                'condition' : conditions,
                'transmission' : transmissions,  
                'description' : descriptions,
                'page_link' : page_link
            })  
    
    return dataframe

In [ ]:
dataframe = data_collecte(all_car_links)

.

Thanks for looking through the notebook.

If you have a data collection project, do not hesitate to entrust it to me. I will provide you with both the code and the collected dataset.

Gickel OKABI

mail : i.gickelokabi@gmail.com